## Print POD

In [1]:
import os
import re
import win32api
import win32print
from datetime import datetime

# Printing Function
def print_pdf(path):
    try:
        win32api.ShellExecute(0, "print", path, None, ".", 0)
        print(f"✅ Sent to printer: {os.path.basename(path)}")
    except Exception as e:
        print(f"❌ Failed to print {path}: {e}")

# ───────────────────────────────
# 1.  Folder with POD PDFs
# ───────────────────────────────
FOLDER = r"\\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD"

# ───────────────────────────────
# 2.  Copy-&-paste POD list here
# ───────────────────────────────
raw_input = """
POD-251603,POD-251717,POD-251722,POD-251723,POD-251730,POD-251740,POD-251749,POD-251754,POD-251757              
"""

# ───────────────────────────────
# 3.  Parse & numerically sort PODs
# ───────────────────────────────
pod_list = [p.strip() for p in raw_input.replace("\n", "").split(",") if p.strip()]
PODS_ORDERED = sorted(pod_list, key=lambda x: int(x.split("-")[1]))  # ascending 250382 → 250443 …
PODS_SET     = set(PODS_ORDERED)   # fast membership test


# Track latest file per POD
latest_signed_files = {}

for pod in PODS_SET:
    latest_time = None
    latest_file = None

    for filename in os.listdir(FOLDER):
        if pod in filename:
            full_path = os.path.join(FOLDER, filename)
            modified_time = os.path.getmtime(full_path)

            if latest_time is None or modified_time > latest_time:
                latest_time = modified_time
                latest_file = full_path

    if latest_file:
            latest_signed_files[pod] = latest_file
            print(f"✅ Latest for {pod}: {latest_file} (Modified: {datetime.fromtimestamp(latest_time)})")

            print_pdf(latest_file) # Printing
    else:
        print(f"❌ No signed version found for {pod}")

✅ Latest for POD-251754: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251754_NTA_Inventory.pdf (Modified: 2025-12-22 05:46:23.535980)
✅ Sent to printer: POD-251754_NTA_Inventory.pdf
✅ Latest for POD-251722: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251722_NTA_Inventory.pdf (Modified: 2025-12-12 00:16:31.703119)
✅ Sent to printer: POD-251722_NTA_Inventory.pdf
✅ Latest for POD-251717: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251717_NTA_Neteon_SO-20251725.pdf (Modified: 2025-12-10 22:30:54.996436)
✅ Sent to printer: POD-251717_NTA_Neteon_SO-20251725.pdf
✅ Latest for POD-251603: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Purchase Order (Vendor)- POD\POD-251603_NTA_Brunswick_SO-20251608.pdf (Modified: 2025-11-13 02:12:51.282876)
✅ Sent to printer: POD-251603_NTA_Brunswick_SO-20251608.pdf
✅ Latest for POD-251749: \\Quickbook2024\d\Drive D\QuickBooks\3- Year 2025\Pur

# Tariff Calculator

In [12]:
import pdfplumber
import pandas as pd
import re
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
from tabulate import tabulate

# === TARIFF Model ===
def calculate_tariff(hs_code: str, origin: str) -> float:
    hs_code = hs_code.replace('.', '').strip()
    origin = origin.strip().upper()

    if hs_code.startswith('847141'):
        return 0.0
    elif hs_code.startswith('847330'):
        return 45.0 if origin == 'CN' else 0.0
    elif hs_code.startswith('850440'):
        return 20.0 if origin == 'TW' else 45.0 if origin == 'CN' else 0.0
    elif hs_code.startswith('854442'):
        return 22.6 if origin == 'TW' else 47.6 if origin == 'CN' else 0.0
    elif hs_code.startswith('852910'):
        return 20.0 if origin == 'TW' else 30.0 if origin == 'CN' else 0.0
    elif hs_code.startswith('852589'):
        return 20.0 if origin == 'TW' else 45.0 if origin == 'CN' else 0.0
    return 0.0

# === BLOCK PARSER ===
def parse_item_block(block_lines):
    first_line = block_lines[0]
    item_name = None
    amount = None
    country = "NA"
    hs_code = "NA"
    htsus = "NA"

    match = re.search(r'POD-\d{6,7}[A-Z()]*\s+(.*?)\s+\d+\s+PCS\s+([\d,]+)', first_line)
    if match:
        raw_item = match.group(1).strip()
        item_name = re.sub(r'\s+\d+$', '', raw_item)
        amount = match.group(2).replace(",", "")
    else:
        return None

    for line in block_lines:
        country_match = re.search(r"\b([A-Z]{2})\s*$", line)
        if country_match:
            country = country_match.group(1)
        if "HS" in line.upper():
            hs_match = re.search(r"HS\s*CODE\s*[:：]?\s*(\d+)", line, re.IGNORECASE)
            if hs_match:
                hs_code = hs_match.group(1)
        if "HTSUS" in line:
            htsus_match = re.search(r"HTSUS\s*[:：]?\s*([\d\.]+)", line)
            if htsus_match:
                htsus = htsus_match.group(1)

    tariff = calculate_tariff(hs_code, country)

    return {
        "Item": item_name,
        "Amount": amount,
        "Country of Origin": country,
        "HS Code": hs_code,
        "HTSUS": htsus,
        "Tariff (%)": tariff
    }

# === PDF PARSER ===
def extract_items_from_invoice(pdf_path):
    items = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            lines = page.extract_text().split('\n') if page.extract_text() else []
            current_block = []
            for line in lines:
                if 'PCS' in line:
                    if current_block:
                        item = parse_item_block(current_block)
                        if item:
                            items.append(item)
                    current_block = [line]
                else:
                    if current_block:
                        current_block.append(line)
            if current_block:
                item = parse_item_block(current_block)
                if item:
                    items.append(item)

    df = pd.DataFrame(items)
    if not df.empty:
        df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)
        df['Tariff Amount'] = df['Amount'] * df['Tariff (%)'] / 100
        total_tariff = df['Tariff Amount'].sum()
        service_fee = df['Amount'].sum() * 0.003464
        service_fee = min( service_fee, 651.50)   # max service fee is 634.62
        total_fee = total_tariff + service_fee
        return df, total_tariff, service_fee, total_fee
    return df, 0.0, 0.0, 0.0

# === EXCEL WRITER ===
def write_tariff_to_excel(df, total_tariff, service_fee, total_fee, output_path):
    df.to_excel(output_path, index=False)

    wb = load_workbook(output_path)
    ws = wb.active

    start_row = ws.max_row + 2
    labels = ["TOTAL TARIFF", "MERCHANDISE SERVICE FEE", "TOTAL FEE"]
    values = [total_tariff, service_fee, total_fee]
    fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    for i, (label, value) in enumerate(zip(labels, values)):
        row = start_row + i
        ws.cell(row=row, column=1, value=label).font = Font(bold=True)
        ws.cell(row=row, column=1).fill = fill
        cell = ws.cell(row=row, column=2, value=value)
        cell.font = Font(bold=True)
        cell.number_format = '"$"#,##0.00'
        cell.fill = fill

    wb.save(output_path)
    print(f"✅ Full tariff table with summary saved to: {output_path}")

# === USAGE ===
pdf_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\INVOICE_NTA_8000155.pdf"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\06 Payment\Entry Summary(Template).xlsx"

df_items, total_tariff, service_fee, total_fee = extract_items_from_invoice(pdf_path)
# write_tariff_to_excel(df_items, total_tariff, service_fee, total_fee, output_path)
print(f'Total Items: {len(df_items)}')

print("\n" + "=" * 60)
print("\033[1;44m\033[1;37m     🚢  TARIFF CALCULATION SUMMARY  📦     \033[0m")
print("=" * 60)

print(f"\033[1;33m🔶 Total Tariff :\033[0m   \033[1m${total_tariff:,.2f}\033[0m")
print(f"\033[1;36m🔷 Service Fee  :\033[0m   \033[1m${service_fee:,.2f}\033[0m")
print(f"\033[1;32m🟩 Entry Fee    :\033[0m   \033[1m${total_fee:,.2f}\033[0m")

print("=" * 60 + "\n")

print("\033[1;34m📄 Tariff Breakdown Table:\033[0m")
print(tabulate(df_items, headers='keys', tablefmt='fancy_grid', showindex=True))


Total Items: 18

     🚢  TARIFF CALCULATION SUMMARY  📦     
🔶 Total Tariff :   $7.23
🔷 Service Fee  :   $95.40
🟩 Entry Fee    :   $102.63

📄 Tariff Breakdown Table:
╒════╤════════════════════════════════════════╤══════════╤═════════════════════╤═══════════╤════════════╤══════════════╤═════════════════╕
│    │ Item                                   │   Amount │ Country of Origin   │   HS Code │ HTSUS      │   Tariff (%) │   Tariff Amount │
╞════╪════════════════════════════════════════╪══════════╪═════════════════════╪═══════════╪════════════╪══════════════╪═════════════════╡
│  0 │ Length : 180CM 16.00                   │       32 │ TW                  │    854442 │ NA         │         22.6 │           7.232 │
├────┼────────────────────────────────────────┼──────────┼─────────────────────┼───────────┼────────────┼──────────────┼─────────────────┤
│  1 │ 7425.00                                │    14850 │ TW                  │    847150 │ 9903.01.32 │          0   │           0     │
├

# Packinglist Extraction

In [13]:
PL_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\PL_8000155_NTA.pdf"

In [14]:
with pdfplumber.open(PL_path) as pdf:
    for page in pdf.pages:
        print(page.extract_text())

Packing List
Neousys Technology Inc. INV NO: 8000155
11F., No. 198, Jian 8th Rd., Zhonghe Dis INV Date: Jan. 12, 2026
New Taipei City 235042, Taiwan Incoterm: CPT
TEL: +886-2-22236182 Shipping Method: Air
FAX: +886-2-22236183 Speedmark
Bill To: Ship To:
Neousys Technology America Inc. Neousys Technology America Inc.
55 East Hintz Rd., Wheeling, IL 55 East Hintz Rd., Wheeling, IL
60090 USA 60090 USA
Phone: Phone:
ATTN: Roy Wang ATTN: Roy Wang
CARTON No MODEL NAME / Description QTY Net Weight Gross Weight Dimension (CM)
1 MezIO-V20 2 0.10 3.40 36 X 29 X 28
16-mode ignition power control and 1x mini-PCIe socket (U2 only) MezIO m odule
for Nuvo LP and POC series
Customer PO No.: POD-251740
PCIe-10G550X 6 0.99
2-port 10GbE Network Adapter
Customer PO No.: POD-251754
PCIe-PoE352at 3 0.77
2-port Intel® I350-AM2 server-grade Gigabit IEEE 802.3at PoE+ frame grab ber
card
Customer PO No.: POD-251740
AccsyBx-Nuvo-9000E/P series 1 0.50
Accessory box kit for Nuvo-9000E/P series, including 2xWall Mo

In [27]:
import pdfplumber
import re
import pandas as pd

pdf_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\PL_8000155_NTA.pdf"

# --- Signals / Patterns ---
PO_LINE_RE = re.compile(r"^Customer PO No\.\s*:\s*(POD-\d+)\s*$", re.IGNORECASE)
TOTAL_RE = re.compile(r"^TOTAL\b", re.IGNORECASE)

# Main item line patterns (we only need carton/model/qty)
# With carton: "2-3 <MODEL NAME ...> 2 0.24 ..."
ITEM_WITH_CARTON_RE = re.compile(
    r"^(?P<carton>\d+(?:-\d+)?)\s+(?P<model>.+?)\s+(?P<qty>\d+)\s+(?P<net>\d+\.\d+)\b"
)

# No carton: "<MODEL NAME ...> 6 0.99"
ITEM_NO_CARTON_RE = re.compile(
    r"^(?P<model>.+?)\s+(?P<qty>\d+)\s+(?P<net>\d+\.\d+)\b"
)



def parse_block_for_item(block_lines):
    for ln in block_lines:
        s = ln.strip()
        if not s:
            continue
        m = ITEM_WITH_CARTON_RE.match(s)
        if m:
            return m.group("carton"), m.group("model"), int(m.group("qty"))

    for ln in block_lines:
        s = ln.strip()
        if not s:
            continue
        m = ITEM_NO_CARTON_RE.match(s)
        if m:
            return None, m.group("model"), int(m.group("qty"))

    return None, None, None



rows = []
current_block = []
last_carton = None

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        for line in text.split("\n"):
            line = line.strip()
            if not line:
                continue

            # Stop on TOTAL
            if TOTAL_RE.match(line):
                current_block = []
                break

            # Separator signal: flush block when we hit Customer PO No
            po_m = PO_LINE_RE.match(line)
            if po_m:
                pod = po_m.group(1).upper()

                carton, model, qty = parse_block_for_item(current_block)

                if model is not None:
                    if carton:
                        last_carton = carton
                        carton_out = last_carton if last_carton else "NA"
                    rows.append({
                        "CARTON No": carton_out,
                        "MODEL NAME": model,
                        "QTY": qty if qty is not None else 0,
                        "POD": pod
                    })

                current_block = []  # reset for next item
                continue

            # Otherwise keep accumulating
            current_block.append(line)

# Build DataFrame
df_packing = pd.DataFrame(rows, columns=["CARTON No", "POD", "MODEL NAME", "QTY"])

print(f"Total Items: {len(df_packing)}")
print(df_packing)



Total Items: 18
   CARTON No         POD                               MODEL NAME  QTY
0          1  POD-251740                                MezIO-V20    2
1          1  POD-251754                             PCIe-10G550X    6
2          1  POD-251740                            PCIe-PoE352at    3
3          1  POD-251749              AccsyBx-Nuvo-9000E/P series    1
4        2-3  POD-251603                   Cbl-M12A17M-VGA-180CM3    2
5        2-3  POD-251603                 SEMIL-1748GC-10G-L4-EL06    2
6        4-5  POD-251722                       Nuvo-9006DE-PoE-UL    4
7          6  POD-251717                           Nuvo-11531-IGN    1
8          6  POD-251723                               POC-551VTC    3
9          6  POD-251723                       Wmkit-V-POC300_400   10
10         6  POD-251749                        Nuvo-9000 Chassis    1
11         6  POD-251730  AccsyBx-Cardholder-10108GC-5080_70_70Ti    1
12         6  POD-251754                                  POC

## Generate Receiving Log (Expend by Qty)

In [37]:
import pandas as pd

# List of prefixes to skip expansion
skip_prefixes = ("Cbl", "DIN", "AccsyBx", "Gpubr", "Ant", "FK", "TB", "Screw", "Wmkit", "Dmpbr")

# Expected columns in df_packing:
# ["CARTON No", "MODEL NAME", "QTY", "Customer PO No"]

# Normalize string columns
df_packing["MODEL NAME"] = df_packing["MODEL NAME"].astype(str).str.strip()
df_packing["CARTON No"] = df_packing["CARTON No"].astype(str).str.strip()

# Add flag for whether to expand
df_packing["Expand"] = ~df_packing["MODEL NAME"].str.startswith(skip_prefixes)

# Store original row order
df_packing["OriginalIndex"] = df_packing.index

expanded_rows = []

for _, row in df_packing.iterrows():
    qty = int(row["QTY"]) if pd.notna(row["QTY"]) else 0
    carton = row["CARTON No"]
    part = row["MODEL NAME"]
    pod = row["POD"]

    if row["Expand"]:
        for _ in range(qty):
            expanded_rows.append({
                "Box No.": carton,
                "Product Number": part,
                "Qty": pd.NA,              # blank for expanded rows
                "OriginalIndex": row["OriginalIndex"],
                "Customer PO No": pod
            })
    else:
        expanded_rows.append({
            "Box No.": carton,
            "Product Number": part,
            "Qty": qty,                  # keep integer
            "OriginalIndex": row["OriginalIndex"],
            "Customer PO No": pod
        })

final_df = pd.DataFrame(expanded_rows)

# Ensure Qty is nullable integer
final_df["Qty"] = final_df["Qty"].astype("Int64")

# Restore original order
final_df.sort_values("OriginalIndex", inplace=True)
final_df.drop(columns=["OriginalIndex", "Expand"], errors="ignore", inplace=True)
final_df.reset_index(drop=True, inplace=True)

# Reformat to your template
final_df["Box #"] = final_df["Box No."]
final_df["Part#"] = final_df["Product Number"]
final_df["POD#"] = final_df["Customer PO No"].fillna("")
final_df["SN#"] = ""  # blank

# Final column order
output_df = final_df[["Box #", "POD#", "Part#", "SN#", "Qty"]]

export_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Invoice\EXPORT_8000155_NTA.xlsx"

# 1) Read export file (the one in your screenshot)
df_export = pd.read_excel(export_path, dtype=str)

# Normalize column names if needed (adjust if your headers differ)
# Expected columns: "Customer PO No", "Ordering Model", "S/N"
df_export["Customer PO No"] = df_export["Customer PO No"].astype(str).str.strip()
df_export["Ordering Model"] = df_export["Ordering Model"].astype(str).str.strip()
df_export["S/N"] = df_export["S/N"].astype(str).str.strip()

# 2) Build a serial-number pool per (PO, Model)
# Example: {("POD-251603","SEMIL-..."): [sn1, sn2, ...], ...}
sn_pool = (
    df_export.groupby(["Customer PO No", "Ordering Model"])["S/N"]
    .apply(list)
    .to_dict()
)

# 3) Normalize output_df keys
output_df["POD#"] = output_df["POD#"].astype(str).str.strip()
output_df["Part#"] = output_df["Part#"].astype(str).str.strip()

# Optional: only assign SN to rows that represent 1 unit (your expanded rows have Qty = NA)
# If you want to assign SN to ALL rows regardless, remove this mask.
mask_assign = output_df["SN#"].astype(str).str.strip().eq("") & output_df["Qty"].isna()

# 4) Assign SN sequentially per key
used_count = {}  # how many SN already used per (PO, Model)

def pick_sn(po, model):
    key = (po, model)
    if key not in sn_pool:
        return ""
    idx = used_count.get(key, 0)
    if idx >= len(sn_pool[key]):
        return ""  # not enough SNs
    used_count[key] = idx + 1
    return sn_pool[key][idx]

# Fill SN# (row-by-row so assignment is sequential)
for i in output_df.index[mask_assign]:
    po = output_df.at[i, "POD#"]
    model = output_df.at[i, "Part#"]
    output_df.at[i, "SN#"] = pick_sn(po, model)

# Save to Excel
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Desktop\Output.xlsx"
output_df.to_excel(output_path, index=False)

print("Total Items:", len(output_df))
print("\n📦 FULL PACKING LIST SECTION:")
print(output_df)

Total Items: 45

📦 FULL PACKING LIST SECTION:
   Box #        POD#                                    Part#  \
0      1  POD-251740                                MezIO-V20   
1      1  POD-251740                                MezIO-V20   
2      1  POD-251754                             PCIe-10G550X   
3      1  POD-251754                             PCIe-10G550X   
4      1  POD-251754                             PCIe-10G550X   
5      1  POD-251754                             PCIe-10G550X   
6      1  POD-251754                             PCIe-10G550X   
7      1  POD-251754                             PCIe-10G550X   
8      1  POD-251740                            PCIe-PoE352at   
9      1  POD-251740                            PCIe-PoE352at   
10     1  POD-251740                            PCIe-PoE352at   
11     1  POD-251749              AccsyBx-Nuvo-9000E/P series   
12   2-3  POD-251603                   Cbl-M12A17M-VGA-180CM3   
13   2-3  POD-251603                 SEMIL-1

C:\Users\Admin\AppData\Local\Temp\ipykernel_7976\617304242.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["POD#"] = output_df["POD#"].astype(str).str.strip()
C:\Users\Admin\AppData\Local\Temp\ipykernel_7976\617304242.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["Part#"] = output_df["Part#"].astype(str).str.strip()


## Upload receiving log to DB

In [18]:
import logging
from flask import Flask, jsonify # type: ignore
from flask_sqlalchemy import SQLAlchemy # type: ignore
import pandas as pd # type: ignore
from sqlalchemy import Column, String, Float, Date, Integer



def load_receiving_log(path_to_xlsm: str, engine, dry_run=False):
    import pandas as pd

    # Load only the 'Receiving' sheet
    df = pd.read_excel(path_to_xlsm, sheet_name="Receiving")

    # Remove rows where all critical fields are blank (before renaming)
    df = df.dropna(subset=['Date', 'Inv# ', 'Box #', 'POD#', 'Part#', 'SN#'], how='all')

    # Rename columns to standardized names
    df.rename(columns={
        'Date': 'entry_date',
        'Inv# ': 'invoice_number',
        'Box #': 'box_number',
        'POD#': 'pod_number',
        'Part#': 'part_number',
        'SN#': 'serial_number',
        'QTY': 'quantity'
    }, inplace=True)

    # Normalize types and strip spaces
    df['quantity'] = df['quantity'].fillna(1).astype(float)
    df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')
    df['serial_number'] = df['serial_number'].astype(str).str.strip().replace("nan", "NA")

    string_cols = ['invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number']
    for col in string_cols:
        df[col] = df[col].astype(str).str.strip()

    # Print preview after cleanup
    print(f"🧾 Cleaned DataFrame Preview:\n{df.tail(25)}")

    # Load existing data for deduplication
    existing = pd.read_sql("""
        SELECT entry_date, invoice_number, box_number, pod_number, part_number, serial_number, quantity
        FROM receiving_log
    """, engine)

    existing['entry_date'] = pd.to_datetime(existing['entry_date'], errors='coerce')
    for col in string_cols:
        existing[col] = existing[col].astype(str).str.strip()
    existing['quantity'] = existing['quantity'].astype(float)


    # 🔍 Deduplicate based on key fields
    key_cols = ['entry_date', 'invoice_number', 'box_number', 'pod_number', 'part_number', 'serial_number', 'quantity']
    merged = df.merge(existing, how='left', indicator=True, on=key_cols) ## Only if all the seven columns are matched, otherwise we take it as new line
    print(f"🧾 merged:\n{merged.tail(25)}")
    new_rows = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])

    print(f"🟡 Dry Run: {len(new_rows)} new rows would be inserted (out of {len(df)} total).")

    if not dry_run:
        new_rows.to_sql('receiving_log', engine, if_exists='append', index=False, method='multi')
        print("✅ Data inserted.")
    else:
        print("🚫 Dry run mode — no data inserted.")
        print("🔍 Preview of rows to be inserted:")
        print(new_rows.head(10))


# Configure Supabase
logging.basicConfig(level=logging.INFO)

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False


db = SQLAlchemy()

# Hook your existing db object up to this app
db.init_app(app)

class ReceivingLog(db.Model):
    __tablename__   = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number   = db.Column(db.String(255), primary_key=True)
    entry_date      = db.Column(db.Date)
    invoice_number  = db.Column(db.String(255))
    box_number      = db.Column(db.String(255))
    pod_number      = db.Column(db.String(255))
    part_number     = db.Column(db.String(255))
    quantity        = db.Column(db.Float)
    reference     = db.Column('Reference', db.Text)  


if __name__ == '__main__':
    # 1️⃣ Push the Flask context so db.engine is wired up:
    with app.app_context():
        # 2️⃣ Ensure the receiving_log table exists
        db.create_all()

        # 3️⃣ Now call your loader, handing it db.engine
        load_receiving_log(
            r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Receiving Log_ZC_2.0.xlsm",
            db.engine
        )

🧾 Cleaned DataFrame Preview:
     entry_date invoice_number box_number  pod_number   part_number  \
6964 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6965 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6966 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6967 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6968 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6969 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6970 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6971 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6972 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6973 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6974 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6975 2025-12-23    IN251217002  74- 68-72  POD-251682   PA-280W-ET3   
6976 2025-12-23    IN251217002  74- 68-72  POD-2

## Generate Incoming Form

In [19]:
import logging
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Column, String, Float, Date, Integer


# Configure Flask app and SQLAlchemy
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:"
    "Czheyuan0227@aws-0-us-east-2.pooler.supabase.com:6543/postgres"
)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define the model
class ReceivingLog(db.Model):
    __tablename__ = 'receiving_log'
    id = Column(Integer, primary_key=True, autoincrement=True)
    serial_number = Column(String(255))
    entry_date    = db.Column(db.Date)
    invoice_number= db.Column(db.String(255))
    box_number    = db.Column(db.String(255))
    pod_number    = db.Column(db.String(255))
    part_number   = db.Column(db.String(255))
    quantity      = db.Column(db.INT)
    reference      = db.Column('Reference', db.Text) 

# Query the database inside the app context
with app.app_context():
    rows = ReceivingLog.query.filter_by(invoice_number='IN251217002').all()

    df = pd.DataFrame([{
        "Part Number": r.part_number,
        "Quantity": r.quantity,
        "Serial Number": r.serial_number,
        "Box Number": r.box_number,
        "POD Number": r.pod_number,
        
    } for r in rows])

    df['Serial Number'] = df['Serial Number'].fillna("NA").astype(str).str.strip()
    
# Step 1: Group and aggregate Quantity, preserving order
df_reset = df.reset_index()
grp = (
    df_reset
    .groupby(["Box Number", "Part Number", "POD Number"], as_index=False)
    .agg(
        Quantity=("Quantity", "sum"),
        first_idx=("index", "min")
    )
    .sort_values("first_idx")
    .drop(columns="first_idx")
    .reset_index(drop=True)
)

# Step 2: Build a serial number map per group (ignore exact duplicates, but keep 'NA')
serial_map = (
    df.groupby(["Box Number", "Part Number", "POD Number"])["Serial Number"]
    .apply(lambda x: ", ".join(x.astype(str).dropna().unique()))
    .reset_index()
    .rename(columns={"Serial Number": "Serial Numbers"})
)

# Step 3: Merge serial numbers into the main grouped result
grp_with_serials = grp.merge(
    serial_map,
    on=["Box Number", "Part Number", "POD Number"],
    how="left"
)

# Step 4: Reorder columns (optional)
grp_with_serials = grp_with_serials[
    ["Part Number", "Quantity", "Serial Numbers", "Box Number", "POD Number"]
]

print(f'Total Lines:', len(grp_with_serials))

# Step 5: Display in Jupyter notebook
from IPython.display import display, HTML
display(HTML(grp_with_serials.to_html(index=False)))



Total Lines: 56


Part Number,Quantity,Serial Numbers,Box Number,POD Number
SEMIL-2047GC-i9IC-65W-MAP02-25,37,"P5000246, P5000247, P5000248, P5000249, P5000250, P5000251, P5000252, P5000253, P5000254, P5000255, P5000256, P5000257, P5000258, P5000259, P5000260, P5000261, P5000262, P5000263, P5000264, P5000265, P5000266, P5000267, P5000268, P5000269, P5000270, P5000271, P5000272, P5000273, P5000274, P5000275, P5000276, P5000277, P5000278, P5000279, P5000280, P5000281, P5000282",74- 1-37,POD-251446
Nuvo-7208VTC,12,"P5000011, P5000012, P5000013, P5000014, P5000015, P5000016, P5000017, P5000018, P5000019, P5000020, P5000021, P5000022",74- 38-43,POD-251668
Nuvo-7166GC-PoE,1,P5000091,74- 44,POD-251650
Nuvo-7208VTC,1,P5000023,74- 44,POD-251668
Cbl-YT-OW-Black-50CM1,50,NA,74- 45,POD-251608
Cbl-YT-OW-Red-50CM1,50,NA,74- 45,POD-251608
Cbl-M12A8M-OW-22CM,10,NA,74- 45,POD-251631
Cbl-M12X8M-RJ45-CAT6A-102CM,10,NA,74- 45,POD-251631
Cbl-M12L5F-OW5-40CM,10,NA,74- 45,POD-251631
RPM-450,3,"P5000430, P5000431, P5000432",74- 45,POD-251643


In [20]:
from docx import Document
from docx.shared import Pt

def write_table_to_word(df, doc_path, output_path):
    # Load the existing Word document
    doc = Document(doc_path)

    # Add a new table with headers + rows
    table = doc.add_table(rows=1, cols=len(df.columns))
    # table.style = 'Table Grid'

    # Write the header row
    hdr_cells = table.rows[0].cells
    for i, column_name in enumerate(df.columns):
        hdr_cells[i].text = column_name

    # Write the data rows
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)

    doc.add_paragraph()

    # Add the result line at the bottom
    result_paragraph = doc.add_paragraph("[Receiving & Inspection Result: Pass]")
    run = result_paragraph.runs[0]
    run.bold = True
    run.font.size = Pt(14)   
    run.font.name = 'Calibri' 
    # Save to new Word file
    doc.save(output_path)
    print("✅ New table written to:", output_path)

# Example usage:
doc_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\Incoming Form Template.docx"
output_path = r"C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx"
write_table_to_word(grp_with_serials, doc_path, output_path)



✅ New table written to: C:\Users\Admin\OneDrive - neousys-tech\Share NTA Warehouse\01 Incoming\I07-070125-TaiwanHQ-IN240628006(Template).docx
